In [ ]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import cloudscraper
import time
import random

In [ ]:
years = list(range(2026, 2019, -1))
years

In [ ]:
all_matches = []

In [ ]:
standings_url = "https://fbref.com/en/comps/12/La-Liga-Stats"

In [ ]:
# Dodaj te nagłówki
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36',
    'Accept-Language': 'en-US,en;q=0.9',
    'Referer': 'https://www.google.com/',
    'Connection': 'keep-alive'
}

# Wewnątrz pętli użyj ich w scraperze:
scraper = cloudscraper.create_scraper()
data = scraper.get(standings_url, headers=headers) # <--- Przekazanie headers

In [ ]:
for year in years:
    scraper = cloudscraper.create_scraper()
    data = scraper.get(standings_url, headers=headers)
    
    # --- DIAGNOSTYKA ---
    print(f"Pobieram: {standings_url}")
    print(f"Status Code: {data.status_code}") # 200 = OK, 429 = Za dużo zapytań, 403 = Zabronione
    
    if data.status_code != 200:
        print("Zostałeś zablokowany! Treść odpowiedzi:")
        print(data.text[:500]) # Pokaż pierwsze 500 znaków błędu
        break # Przerwij pętlę, nie ma sensu dalej próbować
    # -------------------

    soup = BeautifulSoup(data.text, 'html.parser')
    
    # Dodaj zabezpieczenie przed pustą listą
    tables = soup.select('table.stats_table')
    if not tables:
        print(f"Nie znaleziono tabeli dla roku {year}. Być może struktura strony się zmieniła lub to strona błędu.")
        continue
        
    standings_table = soup.select('table.stats_table')[0]

    links = [l.get("href") for l in standings_table.find_all('a')]
    links = [l for l in links if '/squads/' in l]
    team_urls = [f"https://fbref.com{l}" for l in links]

    previous_season = soup.select("a.prev")[0].get("href")
    standings_url = f"https://fbref.com{previous_season}"
    time.sleep(random.uniform(10, 15))

    for team_url in team_urls:
        team_name = team_url.split("/")[-1].replace("-Stats", "").replace("-", " ")

        time.sleep(random.uniform(10, 15))
        
        data = scraper.get(team_url, headers=headers)
        matches = pd.read_html(data.text, match="Scores & Fixtures")[0]
        soup = BeautifulSoup(data.text, 'html.parser')
        links = [l.get("href") for l in soup.find_all('a')]
        links = [l for l in links if l and 'all_comps/shooting/' in l]
        
        time.sleep(random.uniform(10, 15))
        
        data = scraper.get(f"https://fbref.com{links[0]}", headers=headers)
        shooting = pd.read_html(data.text, match="Shooting")[0]
        shooting.columns = shooting.columns.droplevel()
        
        try:
            team_data = matches.merge(shooting[["Date", "Sh", "SoT", "Dist", "FK", "PK", "PKatt"]], on="Date")
        except ValueError:
            continue
        team_data = team_data[team_data["Comp"] == "La Liga"]

        team_data["Season"] = year
        team_data["Team"] = team_name
        all_matches.append(team_data)
        time.sleep(random.uniform(30, 60))

In [ ]:
match_df = pd.concat(all_matches)

In [ ]:
match_df.columns = [c.lower() for c in match_df.columns]

In [ ]:
match_df.to_csv("matches.csv")